In [26]:
import numpy as np
import matplotlib as plt

In [27]:
class UCB:
  def __init__(self, c=1.414):  #n is number of arms
    """The arms that we have decided is that we will have pxx,pxy,pyx,pyy all these are filters of num_taps=3 and the number of values for each tap will be 3("the best we find from lms algorithm")"""

    """NOTE: Wont selecting the best 3 arms from lms algorithm bias the UCB algorithm?.What is the point of this?.Also the number of arms here will be 3^12"""

    """TODO: Find out the top 3 values from lms algorithm for each of the 12 coefficients"""

    #first 3 are pxx then pxy,pyx,pyy
    set_of_coefficients = np.array([[1, 1, 1], [1, -1, -1], [1, 1, -1], [1, -1, 1], [1, 1, 1], [1, -1, -1], [1, 1, -1], [1, -1, 1], [1, 1, 1], [1, -1, -1], [1, 1, -1], [1, -1, 1]])
    
    #TODO: Get input symbols from the matlab code file
    self.input_symbols_x_pol = np.array([[1, 3],[2,4],[-1,1],[1, 3],[2,4],[-1,1]])
    self.input_symbols_y_pol = np.array([[1, 3],[2,4],[-1,1],[1, 3],[2,4],[-1,1]])

    def create_arm_array(set_of_coefficients):
        #I take in the set of coefficients and then create all possible 3^12 combinations
        final_out = []  
        temp=[]
        def add(set_of_coefficients,curr_ind):
            if curr_ind == len(set_of_coefficients):
                #print(temp)
                final_out.append(temp[:])
                #print(final_out)
                return
            
            for i in range(len(set_of_coefficients[0])):
                temp.append(set_of_coefficients[curr_ind][i])
                add(set_of_coefficients,curr_ind+1)
                temp.pop()
            
            return final_out
        return add(set_of_coefficients,0)
    
    self.arms = np.array(create_arm_array(set_of_coefficients))   
    
    self.symbol_basis=[]
    for ele in [1,3]:
       for ele2 in [1,3]:
          self.symbol_basis.append([ele,ele2])
          self.symbol_basis.append([-ele,ele2])
          self.symbol_basis.append([ele,-ele2])
          self.symbol_basis.append([+ele,-ele2])

    self.symbol_basis=np.array(self.symbol_basis)    
    self.filter_lengths=3
    self.cur_ind=self.filter_lengths
    self.n_arm = len(self.arms)                  
    self.arm_avg = np.zeros(len(self.arms))                  #sample mean reward for nth arm
    self.arm_pick = np.ones(len(self.arms))                  # number of times each arm has been picked
    self.c = c
    self.rounds = 0                                 # total number of rounds played till now
    self.rews = []
  
  def find_nearest_basis(self,current_symbol):
    #This returns the nearest basis vector
    return self.symbol_basis[np.argmin(np.sum((self.symbol_basis-current_symbol)**2,axis=1))]



  def convolve(self,input_stream,curr_index,filter):
     #Assuming the filter is causal,then we have to do x[curr_ind]*filter[0]+x[curr_ind-1]*filter[1]+x[curr_ind-2]*filter[2]
    #TODO:Make sure to modify it because both the signal and the filter are actually list of vectors(complex numbers).Im assuming pxx,pxy,etc are real numbers change it later ig.
    
    #TODO: for ind less than len(filter) issues will happen have to handle it.
    output=[0,0]
    for ind in range(len(filter)):
       output+=input_stream[curr_index-ind]*filter[ind]
    
    return np.array(output)
    
    #segment = input_stream[max(0, curr_index - len(filter) + 1):curr_index + 1][::-1]   #In order to make sure that initial values are also taken into account
    
    #return np.sum(segment * filter[-len(segment):])   


  def reward_func(self,arm,current_index):
     #Here arm is a set of 12 elements and i split it into its different components

    pxx=arm[:3]
    pxy=arm[3:6]
    pyx=arm[6:9]
    pyy=arm[9:12]



    predicted_symbol_x=self.convolve(self.input_symbols_x_pol,current_index,pxx)+self.convolve(self.input_symbols_y_pol,current_index,pxy)
    
    predicted_symbol_y=self.convolve(self.input_symbols_y_pol,current_index,pyy)+self.convolve(self.input_symbols_x_pol,current_index,pyx)

    nearest_basis_x=self.find_nearest_basis(predicted_symbol_x)
    nearest_basis_y=self.find_nearest_basis(predicted_symbol_y)
    return np.sum((nearest_basis_x-predicted_symbol_x)**2)+np.sum((nearest_basis_y-predicted_symbol_y)**2)

  def initialise(self):                             #playing each arm once
    self.arm_avg = np.array([self.reward_func(arm,self.filter_lengths) for arm in self.arms])

  def best_arm(self):
    return np.argmax(self.arm_avg + self.c * (np.array([np.log(self.rounds)])/self.arm_pick)**0.5)

  def update(self,rounds=1):
    #I am setting rounds = 1 so that every timestep we play one round
    self.rounds += rounds
    for i in range(rounds):
      arm_index = self.best_arm()
      #print(arm)
      reward = self.reward_func(self.arms[arm_index],self.cur_ind)
      self.cur_ind+=1
      #print(reward)
      # if(arm==6):
      #   print(reward)
      num = ( self.arm_avg[arm_index] * self.arm_pick[arm_index] + reward )
      denom = (self.arm_pick[arm_index] + 1.0)
      self.arm_avg[arm_index] = num / denom
      self.arm_pick[arm_index] += 1
      self.rews.append(reward)

  def plotter(self):
    rews = np.cumsum(self.rews).astype(float)
    for i in range(len(rews)):
      rews[i] = rews[i]/(i+1.0)
    plt.plot(range(1,len(rews)+1),rews)

In [28]:
obj=UCB()
obj.initialise()
obj.update(1)
print(obj.best_arm())

0


In [31]:
print(len(obj.arm_avg))

531441
